## Задача: прогноз оттока пользователей с курса по анализу данных в R

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
events_data = pd.read_csv('../datasets/event_data_train.csv')
submission_data = pd.read_csv('../datasets/submissions_data_train.csv')


In [3]:
users_data = pd.pivot_table(index='user_id',\
                            columns='action',\
                            values='step_id',\
                            aggfunc='count',\
                            data=events_data,\
                            fill_value=0).reset_index()

users_data['is_passed_course'] = 0

users_data.loc[users_data['passed'] >= 40,'is_passed_course'] = 1 

is_pass_course = users_data[['user_id','is_passed_course']].copy()

In [4]:
# ### данные за 2 дня 


# #время первого действия
# min_user_timestamp = events_data_all.groupby('user_id',as_index=False)\
#     .agg({'timestamp':'min'})[['user_id','timestamp']]\
#     .rename(columns={'timestamp':'min_timestamp'})

# events_data_all = pd.merge(events_data_all,min_user_timestamp,on='user_id')

# # timestamp которые не выходят дальше 3 дней с момента первого действия
# mask = events_data_all['timestamp'] - events_data_all['min_timestamp'] <= 2 * 24 * 60 * 60

# events_data = events_data_all[mask]


# min_user_sub_timestamp = submission_data_all.groupby('user_id',as_index=False)\
#     .agg({'timestamp':'min'})[['user_id','timestamp']]\
#     .rename(columns={'timestamp':'min_timestamp'})

# submission_data_all = pd.merge(submission_data_all, min_user_sub_timestamp, on='user_id')

# mask = submission_data_all['timestamp'] - submission_data_all['min_timestamp'] <= 2 * 24 * 60 * 60

# submission_data = submission_data_all[mask]


### Предобработка данных

In [5]:
events_data['date'] = pd.to_datetime(events_data['timestamp'],unit='s')
events_data['day'] = events_data['date'].dt.date

#### попробуем прогнозированть отток по поведению пользователя за первые 2 дня 

In [6]:
# выбираем из наблюдений первые 2 дня с момента первого действия пользователя

#время первого действия
min_user_timestamp = events_data.groupby('user_id',as_index=False)\
    .agg({'timestamp':'min'})[['user_id','timestamp']]\
    .rename(columns={'timestamp':'min_timestamp'})

events_data = pd.merge(events_data,min_user_timestamp,on='user_id')

# timestamp которые не выходят дальше 3 дней с момента первого действия
mask = events_data['timestamp'] - events_data['min_timestamp'] <= 2 * 24 * 60 * 60

events_2days = events_data[mask]

In [7]:
# тоже самое с submission_data

min_user_sub_timestamp = submission_data.groupby('user_id',as_index=False)\
    .agg({'timestamp':'min'})[['user_id','timestamp']]\
    .rename(columns={'timestamp':'min_timestamp'})

submission_data = pd.merge(submission_data, min_user_sub_timestamp, on='user_id')

mask = submission_data['timestamp'] - submission_data['min_timestamp'] <= 2 * 24 * 60 * 60

submission_2days = submission_data[mask]

In [50]:
events_2days = pd.read_csv('https://stepik.org/media/attachments/course/4852/events_data_test.csv')
submission_2days = pd.read_csv('https://stepik.org/media/attachments/course/4852/submission_data_test.csv')


In [51]:
pd.unique(events_data['action'])

array(['viewed', 'passed', 'discovered', 'started_attempt'], dtype=object)

In [52]:
users_2days_data = pd.pivot_table(index='user_id',\
                            columns='action',\
                            values='step_id',\
                            aggfunc='count',\
                            data=events_2days,\
                            fill_value=0).reset_index()

users_2days_data.head()

action,user_id,discovered,passed,started_attempt,viewed
0,4,1,1,0,1
1,6,1,1,0,1
2,10,2,2,0,6
3,12,11,9,4,14
4,13,70,70,35,105


In [53]:
sub_tries = submission_2days.groupby('user_id',as_index=False)\
    .agg({'step_id':'count'})[['user_id','step_id']]\
    .rename(columns={'step_id':'steps_treis'})

users_2days_data = users_2days_data.merge(sub_tries, on='user_id')

In [54]:
correct_and_wrong_sub = pd.pivot_table(index='user_id',\
               columns='submission_status',\
               values='step_id',\
               data=submission_2days,\
               aggfunc='count',\
               fill_value=0).reset_index().rename(columns={'correct':'correct_sub','wrong':'wrong_sub'})

correct_and_wrong_sub['correct_ratio'] = correct_and_wrong_sub['correct_sub'] /(correct_and_wrong_sub['correct_sub'] + correct_and_wrong_sub['wrong_sub'])

correct_and_wrong_sub.head()

submission_status,user_id,correct_sub,wrong_sub,correct_ratio
0,12,1,0,1.000000
1,13,29,36,0.446154
2,15,10,30,0.250000
3,21,24,103,0.188976
4,35,7,35,0.166667


In [55]:
users_2days_data = users_2days_data.merge(correct_and_wrong_sub, on='user_id')



#### средний временной промежуток пользователя между началом степа и его выполнением 

In [56]:
# сколько в среднем пользователь тратит времени на решение 

started_attempt = events_2days[events_2days['action'] == 'started_attempt']\
    .groupby(['user_id','step_id'], as_index=False)\
    .agg({'timestamp':'min'})[['user_id','step_id', 'timestamp']].rename(columns={'timestamp': 'start_time'})

passed = events_2days[events_2days['action'] == 'passed']\
    .groupby(['user_id','step_id'], as_index=False)\
    .agg({'timestamp':'min'})[['user_id','step_id', 'timestamp']].rename(columns={'timestamp': 'pass_time'})



In [57]:
sec_b_start_and_pass = pd.merge(started_attempt,passed,on=['user_id','step_id'])
# поскольку timestamp это количество секунд нам нужно перевести, если не перевести будет большая разница между значениями колонок 
# что негативно скажется на предсказаниях модели 

sec_b_start_and_pass['time_margin'] = ((sec_b_start_and_pass['pass_time']- sec_b_start_and_pass['start_time'])/60/60)

b_start_and_pass = sec_b_start_and_pass.groupby('user_id',as_index=False)\
    .agg({'time_margin':'mean'})[['user_id','time_margin']]\
    .rename(columns={'time_margin':'gap_s_p'})

b_start_and_pass['gap_s_p'] = b_start_and_pass['gap_s_p'].round(2)

In [58]:
users_2days_data = pd.merge(users_2days_data, b_start_and_pass, on='user_id')

In [59]:
users_2days_data.head()

,user_id,discovered,passed,started_attempt,viewed,steps_treis,correct_sub,wrong_sub,correct_ratio,gap_s_p
0,12,11,9,4,14,1,1,0,1.000000,0.01
1,13,70,70,35,105,65,29,36,0.446154,0.72
2,21,74,68,70,98,127,24,103,0.188976,0.03
3,35,34,30,11,70,42,7,35,0.166667,0.13
4,45,9,9,2,10,2,2,0,1.000000,0.02


In [60]:
users_2days_data['viewed_ratio'] = users_2days_data['viewed']/ (users_2days_data['viewed'] +
                                                                users_2days_data['passed'] +
                                                                users_2days_data['discovered'] +
                                                                users_2days_data['started_attempt'])

users_2days_data['viewed_ratio'] = users_2days_data['viewed_ratio'].round(2)



In [61]:
count_events = events_2days.groupby('user_id',as_index=False)\
    .agg({'timestamp':'count'})\
    .rename(columns={'timestamp':'count_events'})

users_2days_data = users_2days_data.merge(count_events,on='user_id')


In [62]:
users_2days_data.head()

,user_id,discovered,passed,started_attempt,viewed,steps_treis,correct_sub,wrong_sub,correct_ratio,gap_s_p,viewed_ratio,count_events
0,12,11,9,4,14,1,1,0,1.000000,0.01,0.37,38
1,13,70,70,35,105,65,29,36,0.446154,0.72,0.38,280
2,21,74,68,70,98,127,24,103,0.188976,0.03,0.32,310
3,35,34,30,11,70,42,7,35,0.166667,0.13,0.48,145
4,45,9,9,2,10,2,2,0,1.000000,0.02,0.33,30


In [63]:
users_2days_data.shape

(2363, 12)

In [22]:
users_2days_data.isna().sum().sum()

0

In [23]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

In [64]:
ids = users_2days_data['user_id'].copy()

In [66]:

X = users_2days_data.drop(['user_id'],axis=1)
#y = users_2days_data['is_passed_course']



X.head()

,discovered,passed,started_attempt,viewed,steps_treis,correct_sub,wrong_sub,correct_ratio,gap_s_p,viewed_ratio,count_events
0,11,9,4,14,1,1,0,1.000000,0.01,0.37,38
1,70,70,35,105,65,29,36,0.446154,0.72,0.38,280
2,74,68,70,98,127,24,103,0.188976,0.03,0.32,310
3,34,30,11,70,42,7,35,0.166667,0.13,0.48,145
4,9,9,2,10,2,2,0,1.000000,0.02,0.33,30


In [41]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.3, random_state=17)

In [27]:
import xgboost as xgb

#best_params
# params = {
#     'n_estimators': 400,
#     'max_depth': 3,
#     'max_delta_step': 7,
#     'eta': 0.3}

clf = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc')

params = {'max_delta_step':[3,4,5,6,7],
          'n_estimators':[250,200,150,180],
          'eta':[0.1,0.2,0.3,0.4],
          'max_depth':[3,4,5,6]
}

rand_clf = RandomizedSearchCV(clf,params,n_jobs=-1,cv=7)

In [28]:

rand_clf.fit(X,y)


RandomizedSearchCV(cv=7,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='auc', gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=None...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None

In [29]:
rand_clf.best_params_

{'n_estimators': 150, 'max_depth': 3, 'max_delta_step': 4, 'eta': 0.2}

In [68]:
best_clf = rand_clf.best_estimator_

pred = best_clf.predict_proba(X)

#roc_auc_score(y,pred)

In [70]:
submit = pd.DataFrame(columns=['user_id','is_gone'])

submit['user_id'] = ids
submit['is_gone'] = pred[:,0]

submit.to_csv('sub7.csv')
submit

,user_id,is_gone
0,12,0.767628
1,13,0.000139
2,21,0.000582
3,35,0.722397
4,45,0.756314
...,...,...
2358,26758,0.662505
2359,26770,0.744246
2360,26775,0.000198
2361,26780,0.001308


In [72]:
pd.read_csv('../Downloads/submission_example.csv')

,user_id,is_gone
0,12,0.26
1,13,0.79
2,15,0.12
3,21,0.67
4,35,0.62
...,...,...
6179,7315,0.72
6180,24300,0.35
6181,820,0.88
6182,3649,0.38


In [50]:
from sklearn.ensemble import RandomForestClassifier


clf_forest = RandomForestClassifier(n_estimators=300, n_jobs=-1, 
                            min_samples_leaf=10, min_samples_split=10, 
                            class_weight='balanced')


In [51]:
clf_forest.fit(X_train,y_train)

RandomForestClassifier(class_weight='balanced', min_samples_leaf=10,
                       min_samples_split=10, n_estimators=300, n_jobs=-1)

In [52]:
pred_forest= clf_forest.predict(X_valid)

roc_auc_score(y_valid,pred_forest)

0.7176002665395624

In [47]:
RandomForestClassifier